The notebooks helps in understanding the common painpoints in the Fixed Sample size Testing approach and how group sequential experimentation method helps in overcoming them. 

#### Topic Covered:

1. Pitfalls in Fixed Sample size Testing Approach
2. Introduction to Group Sequential A/B Testing
3. How Group Sequential Analysis handles the Pitfalls of Fixed Sample size Testing Approach
    + Alpha and Beta Spending Functions
    + Test for Efficacy and Futility
4. Designing the Group Sequential Analysis
5. Walkthrough With Demo
6. Reference Resources

### 1. Pitfalls in Fixed Sample Size Approach

In the fixed sample size design approach, we calculate the sample size with the defined paramters expected Delta in Treatment, Alpha and Beta and wait till the experiment reaches the calcuated sample size. 

Most of the cases, when the sample size is large value which may extend the waiting to 1-2 months for getting the idea of whether the experiment is successfull or degrading.

Usually, Product team would be really interested in understanding whether the new experience is improving or degrading the experience for the end consumer. They expect an early result to decide whether to rollout to more users or stop the initiative which would results in cost saving and also reduce the impact.

#### a. Peeking issue in Fixed Sample Size Design:
When we look at the interim stages of the fixed sample design and evalute the performance of the experiment with the prefined Type I error (Alpha). We are actually increasing the False Positive Rate (Alpha/Type I Error)

Lets take an example: We are looking at the experiment performance  as we reach 33%, 66% and 100% of the fixed sample size(300). We are checking whether the treatment performs better than control at 2.5% Type I error at each analysis. 

In [ ]:
install.packages("gsDesign")
library(gsDesign)
gsProbability(k=3,theta=0,n.I = c(100,200,300),a=array(-20,3),b=array(qnorm(0.975),3))

<b> Based on the above results, effectively there are 5.3% chances that the results may be false positive 
whereas we fixed the tolerance level to 2.5% (False Positive Rate/Type I Error/ Alpha). <b>

#### b. Ability to stop the analysis early in either cases of efficacy or futility

Fixed design doesn't help us in identifying 

+ Treatment performing lesser than the control and also will not result in positive benefits 
at the end of experiment. This ability to idenitying the poor performance will help us in saving the efforts 
and resources.

+ Treatment performing better than the control at the statistically significant leve at the early stage.
So that the feature can be rolled out for broader segment




### 2. Introduction to Group Sequential Testing

Group Sequential Testing is part of adaptive designing practice which helps us in 
evaluting the performance as the data flows in and help us in deciding the winner at the early stage. 

This approach provides us the flexibility to perform analysis at different time interval and also ensuring that the 
design parameters are honored throughout the experiment.

### 3. Handling the pitfalls of Fixed Sample Design

#### a. Alpha and Beta Spending Functions:

Group sequential testing leverages the Alpha and Beta Spending Functions to allocate 
the Type I and Type II error for each of the Interim analysis. Usually, They spend less value at the initial stages
of the test and increase over the time.

There are primarily two approaches for Spending the Type I and Type II error:

1. <b>Error Spending Functions:</b> The error spending decides the critical 
   value based on the alpha and beta which has to be spent at the specific interim analysis.
   
   We will be using <b> gamma error spending function </b>
   
   The gamma error spending function (Hwang, Shih, and DeCani 1990) specifies a gamma cumulative 
   error spending function indexed by the gamma parameter . The boundaries created with  are similar to 
   the boundaries from the Pocock method, and the boundaries created with  or  
   are similar to the boundaries from the O’Brien-Fleming method.

<b> Plot illustrate the spending pattern of gamma functions. i.e the lower the gamma value, strigent the alpha and beta spending function </b>
 

In [ ]:
plot(0:100/100, sfHSD(.025, 0:100/100, -4)$spend, type="l", lwd=2,
  xlab="Proportion of information",
  ylab=expression(paste("Cumulative \ ",alpha,"-spending")),
  main="Hwang-Shih-DeCani Spending Functions")

lines(0:100/100, sfHSD(.025, 0:100/100, -2)$spend, lty=2, lwd=2)
lines(0:100/100, sfHSD(.025, 0:100/100, 1)$spend, lty=3, lwd=2)

legend(x=c(.0, .27), y=.025 * c(.8, 1), lty=1:3, lwd=2,legend=c(expression(paste(gamma," = -4")),
expression(paste(gamma," = -2")),expression(paste(gamma," = 1"))))

2. <b> Boundary Family Functions: </b> The functions specifies how big boundary values should be relative to each other and adjust these relative values by a constant multiple to controll overall error rates. The commonly applied boundary family approach uses the wang-Tsiatis family which includes bounds by Pocock and O'Brien and Fleming.
<b> They are generally used with the equally spaced interim analysis </b>

For example: Pocock approach allocates the error spending value equally to each interim analysis. i.e 0.05/3 for each analysis.

<b> Usefull resource </b>
https://support.sas.com/documentation/cdl/en/statug/63033/HTML/default/viewer.htm#statug_seqdesign_sect030.htm



#### b.Test for Efficacy and Futility

In group sequential testing, Error spending functions helps us in defining the boundary for Efficacy (alpha boundary) 
and futility boundary (beta boundary). With the help of this critical value, we can decide 
whether to accept the treatment as winner or control as winner. 

There are cases where business wants not to stop the experiment eventhough the treatment is 
performing lesser than control in the initial stages. Group Sequential testing helps us design the experiment 
to accodomate both to stop <b>(Binding)</b> or continue eventhough the test statistics crossed the futility boundary<b>(Non Binding)</b>.

In [ ]:
# Cumulative proportion of spending planned at each analysis
# In this case, this is also proportion of final observations at each interim
p <- c(.25, .5, .75)
t <- c(.25,.5,.75)
# Cumulative spending intended at each analysis (for illustration)
p * 0.025
n.fix <- nBinomial(p1=.15, p2=.1, beta=.2)
x <- gsDesign(k=4, n.fix=n.fix, beta=.2, sfu=sfLinear, sfupar=c(t,p),
sfl=sfLinear, sflpar=c(t,p))
plot(x, main=expression(paste("Equal ", alpha[i]^{"+"}, (0), " and ",
beta[i](delta), " for each analysis")))

### 4. Designing the Group Sequential Design

We will be leveraging the gsDesign R language Package to design the experiment and the detailed documentation 
on the gsDesign and output is available in the below link. gsDesign helps us in implementing the above 
discussed ideas by choosing the parameters from the defined set.

<b> There are two steps in the process:</b>

1. Designing the Group sequential design using gsDesign
2. Evaluate the performance of the experiment at interim analysis and decide based on the bounday criteria.

<b>Boundary Derivation based on test type (test.type): </b>
There are 6 types of design supported in the gsDesign

+ 1=one-sided - only considering the Efficacy - upper boundary - Alpha Spending 
+ 2=two-sided symmetric - Both Efficacy and Futility boundary - Both defined by same function.
+ 3=two-sided, asymmetric, beta-spending with binding lower bound 
+ 4=two-sided, asymmetric, beta-spending with non-binding lower bound
+ 5=two-sided, asymmetric, lower bound spending under the null hypothesis with binding lower bound
+ 6=two-sided, asymmetric, lower bound spending under the null hypothesis with non-binding lower bound

<b> gsDesign function supports unplanned analysis or changing the timing of the analysis 
by redefining the Interval(n.I) and timing parameters.</b>

In [ ]:
# By default, The design is done for 2 interim and 1 final analysis 
# with default parameters alpha = 0.025 and beta = 0.8

#Calculating the fixed sample size
fixedsize <- nBinomial(p1=.15,p2=.10)

#equally spaced
initialDesign <- gsdesign(n.fix =fixedsize ) 

# changed the design to accodomate the interim analysis at 25% of fixed sample.
changedDesign <- gsdesign(k=4,n.fix =fixedsize, timing =c(0.25,0.33,0.66,1)) 


### 5. Walkthrough Demo
I am using one of the successfully completed Fixed sample design based experimentation 
to show the benefits of Group sequential experimentation. 

One of the challenges we faced during billing lite experimentation was the low traffic in the intra border LATAM billing with out purchase attempts. on average we were getting 5K attempts whereas we needed an approx sample of 100K and we had to wait to 3 weeks to evaluate the treatment performance.

Using the Group Sequential experimentation design we could confirmed the treatments positive performance within the second week itself. This approach would have helped us by making faster decision.

### 6 Usefull Resources

1. gsDesign R Documentations:
   + Package Documentation: https://cran.r-project.org/web/packages/gsDesign/gsDesign.pdf
   + ReferenceManual: (Takes time to load. Better to download local copy)
   https://github.com/subfuzion/gsDesign/blob/master/gsDesign/pkg/gsDesign/inst/doc/gsDesignManual.pdf   
   
   
   
2. PASS Tool Documentaion: https://ncss-wpengine.netdna-ssl.com/wp-content/themes/ncss/pdf/Procedures/PASS/Group-Sequential_Non-Inferiority_Tests_for_the_Difference_of_Two_Proportions-Simulation.pdf



3. Spending Family Functions: https://support.sas.com/documentation/cdl/en/statug/63033/HTML/default/viewer.htm#statug_seqdesign_sect030.htm
   